In [ ]:
from __future__ import division, print_function
import sys
sys.path.append('../')

import numpy as np
import h5py
import matplotlib.pyplot as pl
from os.path import join, split, splitext
from pypllon.parsers import parse_ctx, load_complex_array
from glob import glob
from tools.plot import imsshow

import os
try:
    ID = os.environ['ID']
    BASEDIR = os.environ['BASEDIR']
except KeyError:
    ID = 'M3Fou'
    BASEDIR = '/Users/dsuess/Downloads/Phaselift_2017_13_3/3x3 phaselift/'
   
print(ID, '\n', BASEDIR,'\n')
DIM = int(ID[1])
np.set_printoptions(precision=3)

# labels of the detectors used
DETECTORS_ALL = ['AH', 'BH', 'CH', 'DH', 'EH', 'FH']
DETECTORS = DETECTORS_ALL[1:1 + DIM]
# id of the experiment
outfile = h5py.File('{}.h5'.format(ID), 'a')
try:
    del outfile['RECR']
except KeyError:
    pass
f = outfile.create_group('RECR')

# General information

In [ ]:
import subprocess

def get_git_revision_hash():
    return subprocess.check_output(['git', 'rev-parse', 'HEAD']).split()[0]

f.attrs['COMMITID'] = get_git_revision_hash()
f.attrs['DETECTORS'] = [s.encode() for s in DETECTORS]

# Preparation data

The matrix to be recovered

In [ ]:
# gauss or RECR doesn't matter for this
targetfile = BASEDIR + '/Unitaries/%s.dat' % ID
target = load_complex_array(targetfile)

In [ ]:
imsshow([target.real, target.imag, np.abs(target)])
f['TARGET'] = target
f['TARGETFILE'] = targetfile
outfile.flush()
print(target)

In [ ]:
def load_pvecs(fnames, prepvecs):
    for fname in fnames:
        vecid = split(splitext(fname)[0])[1]
        prepvecs[vecid] = load_complex_array(fname)
        prepvecs[vecid].attrs['FILE'] = fname

    outfile.flush()
    
fnames = glob(BASEDIR + '/Vectors/VRad%i_*.dat' % DIM)
prepvecs = f.create_group('PREPVECS')
print("Number of RECR preparation vectors: %i" % len(fnames))
load_pvecs(fnames, prepvecs)

# Phaselift Raw Measurement Data

In [ ]:
def load_deteff(fname):
    summarydict = parse_ctx(fname)
    return  np.array([summarydict['det_eff'][det] for det in DETECTORS])

def dict_to_rates(dic):
    return np.array([dic.get(det.lower(), 0) for det in DETECTORS])

def load_counts(fname):
    summarydict = parse_ctx(fname)
    rates = np.array([summarydict[det] for det in DETECTORS])

    parent = summarydict['metadata']['parent']
    path_to_raw = join(split(fname)[0], '..', 'raw', parent + '.ctx')
    
    try:
        c = parse_ctx(path_to_raw)
        raw_rates = np.array([dict_to_rates(val) 
                              for key, val in c.items()
                              if key.startswith('count_rates')])

        assert np.all(np.sum(raw_rates, axis=0) == rates)
        return raw_rates, path_to_raw
    except FileNotFoundError:
        return rates[None, :], fname

def vector_to_counts(globpatt):
    matches = glob(globpatt)
    if len(matches) != 1:
        raise IOError("Wrong number of matches %i" % len(matches))
    return load_counts(matches[0])

In [ ]:
deteff_file = BASEDIR + '/data/det_eff/det_eff.txt'

deteff_all = {name: value 
              for name, value in zip(DETECTORS_ALL, np.loadtxt(deteff_file))}
deteff = np.array([deteff_all[key] for key in DETECTORS])

In [ ]:
rawcounts = f.create_group('RAWCOUNTS')
for pvec in f['PREPVECS'].keys():
    globpatt = BASEDIR + '/data/singles/summed_sorted/%s_%s.ctx' % (ID, pvec)
    try:
        counts, fname = vector_to_counts(globpatt)
        rawcounts[pvec] = counts
        rawcounts[pvec].attrs['FILE'] = fname
    except (IOError) as e:
        print(e)

print("Loaded data for {} vectors.".format(len(rawcounts)))
print("First element has shape {}".format(next(iter(rawcounts.values())).shape))
rawcounts.attrs['DETEFF'] = deteff
rawcounts.attrs['DETEFF_FILE'] = deteff_file
outfile.flush()

# Reference Data
## Single Photon Data
Beware: sometimes they are in the wrong order, i.e. singles_1 corresponds to the 5th column

In [ ]:
def get_colcounts(col_nr):
    globpatt = BASEDIR + '/data/singles/summed_sorted/'
    globpatt += '%s_S%i_%.2i.ctx' % (ID, DIM, col_nr)
    matches = glob(globpatt)
    assert len(matches) == 1, "It's actually {} for {}"\
        .format(len(matches), col_nr)
    summarydict = parse_ctx(matches[0])
    return np.array([summarydict[det] for det in DETECTORS]), matches[0]
    
single_counts = f.create_group('SINGLE_COUNTS')
for n in range(1, len(DETECTORS) + 1):
    count, fname = get_colcounts(n)
    # carefull! Sometimes they are the wrong way around!
    index = n - 1
    single_counts[str(index)] = count
    single_counts[str(index)].attrs['FILE'] = fname

In [ ]:
# since they were taken at the same time
single_counts.attrs['DETEFF'] = f['RAWCOUNTS'].attrs['DETEFF']
single_counts.attrs['DETEFF_FILE'] = f['RAWCOUNTS'].attrs['DETEFF_FILE']
outfile.flush()

To check, we plot something proportional to the singles-transfer matrix. Note that we have to transpose counts since the single_counts[i] refer to columns of the transfer matrix.

In [ ]:
counts = np.array([single_counts[str(i)] for i in range(len(DETECTORS))],
                 dtype=np.float64)
counts *= single_counts.attrs['DETEFF']
ax, *_ = imsshow([np.sqrt(counts).T])
pl.colorbar(ax.images[0])
pl.show()

ax, *_ = imsshow([np.abs(target)])
pl.colorbar(ax.images[0])
pl.show()

# Load reference data

In [ ]:
DETECTORS = f.attrs['DETECTORS']
# Average total photon count (for normalization purposes)
tmat_single = np.array([f['SINGLE_COUNTS'][str(i)] for i in range(len(DETECTORS))], dtype=float)
deteff = f['SINGLE_COUNTS'].attrs['DETEFF']
tmat_single = tmat_single * deteff
# axis = 0 since we flip the tmat later
tmat_single /= np.max(np.sum(tmat_single, axis=0))
tmat_single = np.sqrt(tmat_single.T)
f['TMAT_SINGLE'] = tmat_single

Also, load the reconstruction using singles & dips (data missing, fill in in Sec. Dips)

In [ ]:
try:
    recons = load_complex_array(BASEDIR + '/dip_reconstruction/%s_diprecon.dat' % ID)
    f['DIP_RECONSTRUCTED'] = recons
    outfile.flush()
    imsshow([recons.real, recons.imag, np.abs(recons)])
    pl.show()

    imsshow([target.real, target.imag, np.abs(target)])
    pl.show()
except FileNotFoundError:
    print("Dip reconstruction not found")